In [ ]:
#Zaimportowanie wymaganych bibliotek
import os
import numpy as np
import pandas as pd
from pathlib import Path
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, Conv2DTranspose
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.losses import binary_crossentropy
from numpy import zeros,ones,vstack
from numpy.random import randn,randint,normal,random,choice
from numpy.random import default_rng
from IPython.display import clear_output
from math import trunc

from matplotlib import pyplot as plt

from PIL import Image

In [ ]:
#Parametry
VERSION=0.8

#can be cub-cubism or del-delaunay
DATA="cubism"
#DATA="delaunay"


#Number of saved images during training
OUTPUT = 8 # grid of output x output images
# For now there is no margin between images
SAVE_FREQ = 5 #How often to save images
OUTPUT_PATH='output\\new\\' # where to save images
STAT_PATH="output\\statistics\\" # where to save stats
'''
In this use case, our latent space representations are used to
transform more complex forms of raw data (i.e. images, video),
into simpler representations which are "more convenient to process" and analyze.
'''
NOISE_SIZE = 128#Lantent dimention size

EPOCHS = 5000 #Iterations the biger, the longer the model will train
#Batch_size=10
BATCH_SIZE = 64 #number of images in the Batch. Larger Batch → Weak Generalization, Larger Batches → Fewer updates + shifting data → lower computational costs

#AVERAGE=[["Epoch","Accuracy on real","Accuracy on fake"]]
STATS=[["Epoch","batch_t","d_loss","g_loss","acc_real","acc_fake"]]


In [ ]:
if DATA =="delaunay":
    DATASET_NAME="del"
    PATH_TO_DATA='data/delaunay_data_norm.npy'
else:
    DATASET_NAME="cub"
    PATH_TO_DATA='data/cubism_data_norm.npy'

In [ ]:
train_data = np.load(PATH_TO_DATA)
print("(number of images,size_x,size_y,color_channels)")
print(train_data.shape)
if train_data.shape[1]!=train_data.shape[2]:
    print("Something is wrong")

In [ ]:
#size of images
IMG_SIZE = train_data.shape[1] #rows/cols
IMG_CHANNELS = train_data.shape[3] #color channels in our images

In [ ]:
Image.fromarray(((train_data[1]+1)*255/2).astype(np.uint8))

In [ ]:
#discriminator
def build_discriminator(image_shape):
    model = Sequential(name="Discriminator")
    init = RandomNormal(mean=0.0,stddev=0.02)
    #use bias false--> worse performance
    model.add(Conv2D(64, kernel_size=4, strides=2, padding="same",input_shape=image_shape,data_format="channels_last",kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    #was 0.20
    model.add(Dropout(0.30))    

    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same",kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.30))    
    
    model.add(Conv2D(256, kernel_size=4, strides=2, padding="same",kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))    
    model.add(Dropout(0.30))
    
    model.add(Conv2D(512, kernel_size=4, strides=2, padding="same",kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))    
    model.add(Dropout(0.30))
    
    model.add(Conv2D(1024, kernel_size=4, strides=1, padding="valid",kernel_initializer=init))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))#binary clasification

    opt = Adam(learning_rate=0.00002, beta_1=0.5)
    model.compile(loss=binary_crossentropy,optimizer=opt,metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
#generator
#Based on UNSUPERVISED REPRESENTATION LEARNING WITH DEEP CONVOLUTIONAL GENERATIVE ADVERSARIAL NETWORKS
def build_generator(noise_size, channels):
     #weight init
     init = RandomNormal(mean=0.0,stddev=0.02)
     model = Sequential(name="Generator")
     #todo 25 * 25 * 128
     model.add(Dense(2 * 2 * 128, activation="relu",input_dim=noise_size,kernel_initializer=init))
     model.add(LeakyReLU(alpha=0.2))
     # 25, 25, 128
     model.add(Reshape((2, 2, 128)))
     #strides 2 padding valid
     model.add(Conv2DTranspose(1024,kernel_size=4, strides=2, padding="same",kernel_initializer=init))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))    
               
     model.add(Conv2DTranspose(512,kernel_size=4, strides=2, padding="same",kernel_initializer=init))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))    

     model.add(Conv2DTranspose(256,kernel_size=4, strides=2, padding="same",kernel_initializer=init))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))
     
     model.add(Conv2DTranspose(128,kernel_size=4, strides=2, padding="same",kernel_initializer=init))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))        
          
     model.add(Conv2DTranspose(channels, kernel_size=4,strides=2, padding="same",kernel_initializer=init))
     model.add(Activation("tanh"))    

     model.summary()

     return model

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential(name="GAN")
    model.add(generator)
    model.add(discriminator)
    opt = Adam(learning_rate=0.00002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    model.summary()
    return model

In [ ]:
def save_images(epoch,generator, noise):
    generated_images = generator.predict(noise)
    array=np.empty(((IMG_SIZE,IMG_SIZE,IMG_CHANNELS)),float)
    #from (-1,1) to (0,255)
    #generated_images = (generated_images+1)/2*255
    count=0
    for i in range(OUTPUT):
        for j in range(OUTPUT):
            if j==0:
                array_pom=np.array(generated_images[count].reshape(64,64,3,order='C'))
            else:
                array_pom=np.concatenate((array_pom,np.array(
                    generated_images[count].reshape(64,64,3,order='C'))),axis=1)
            count+=1
        if i==0:
            array=array_pom
        else:
            array=np.concatenate((array,array_pom),axis=0)
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
    filename = os.path.join(OUTPUT_PATH, DATA+"_v_%.1f_e_%d.png"%(VERSION,epoch))
    im = Image.fromarray(((array+1)*255/2).astype(np.uint8))
    im.save(filename)

In [ ]:
def get_real_samples(data, n_samples):
    indexes = randint(0, data.shape[0], n_samples)
    X = data[indexes]
    y = ones((n_samples, 1))
    return X, y

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = default_rng().normal(0.0, 1.0, (n_samples, latent_dim))
    X = generator.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y

def smooth_positive_labels(y):
    return y - 0.3 + (random(y.shape) * 0.5)

# randomly flip some labels
def noisy_labels(y, p_flip):
    # determine the number of labels to flip
    n_select = int(p_flip * y.shape[0])
    # choose labels to flip
    flip_ix = choice([i for i in range(y.shape[0])], size=n_select)
    # invert the labels in place
    y[flip_ix] = 1 - y[flip_ix]
    return y

def generate_latent_points(n_samples, latent_dim):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape((n_samples, latent_dim))
    return x_input

In [ ]:
def summarize_performance(epoch, generator, discriminator, dataset, latent_dim, n_samples=100):
    #getting samples and evaluating
    X_real, y_real = get_real_samples(dataset, n_samples)
    _, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n_samples)
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    AVERAGE.append([epoch,acc_real*100,acc_fake*100])
    print('Accuracy on real: %.0f%%, on fake: %.0f%%' % (acc_real*100, acc_fake*100))
    filename = DATASET_NAME+'_model_v_%.1f_e_%d.h5' % (VERSION,epoch)
    generator.save('models\\new\\'+filename,include_optimizer=True)
    return acc_real,acc_fake
#graf plotting
def summarize_performance_2(epoch, generator,stats=STATS):
    st=pd.DataFrame(stats[1:],columns=stats[0])
    pyplot.subplot(2, 1, 1).set_title("Loss")
    pyplot.plot(st.d_loss_r, label='d-real')
    pyplot.plot(st.d_loss_f, label='d-fake')
    pyplot.plot(st.g_loss, label='gen')
    pyplot.title("Loss")
    pyplot.legend()
    pyplot.subplot(2, 1, 2).set_title("Accuracy")
    pyplot.plot(st.acc_real, label='acc-real')
    pyplot.plot(st.acc_fake, label='acc-fake')\
    #todo poprawić ten tytuł
    pyplot.title("Epoch: %d"%(epoch))
    pyplot.legend()
    plt.show()
    
    filename = DATASET_NAME+'_model_v_%.1f_e_%d.h5' % (VERSION,epoch)
    generator.save('models\\new\\'+filename,include_optimizer=True)
    pyplot.savefig(STAT_PATH+"statistics_plot_v_%.1f.png" % (VERSION))
    st.to_csv((STAT_PATH+'stat_avg_'+DATASET_NAME+'_%.1f.csv'%(ver)))
    #np.savetxt(STAT_PATH+'stat_avg_'+dataset+'_%.1f.csv'%(ver), avg, delimiter =',', fmt ='%s')
    pyplot.close()

In [ ]:
#It has to change here
TEMP_ROUNDS=0
TEMP_ROUNDS_2=0
def check_conditions(accuracy_real,accuracy_fake):
    #if both accuraccies is 100% for 5 epochs, than end program
    condition_1=round(accuracy_real,2)==100.00 and round(accuracy_fake,2)==100.00
    #Generator will be fulling discriminator with 100% accuracy for 10 epochs
    condition_2=round(accuracy_fake,2)==100.00
    if condition_1:
        TEMP_ROUNDS+=1
    else:
        TEMP_ROUNDS=0
    if condition_2:
        TEMP_ROUNDS_2+=1
    else:
        TEMP_ROUNDS_2=0
    if TEMP_ROUNDS==5 or TEMP_ROUNDS_2==10:
        return False 
    return True 

In [ ]:
def save_output(avg,loss,ver=VERSION,dataset=DATASET_NAME):
    np.savetxt(STAT_PATH+'stat_avg_'+dataset+'_%.1f.csv'%(ver), avg, delimiter =', ', fmt ='% s')
    np.savetxt(STAT_PATH+'stat_loss_'+dataset+'_%.1f.csv'%(ver), loss, delimiter =', ', fmt ='% s')

In [ ]:
#This model has a tendency to collapse: multiple inputs --> the same output
def train(generator, discriminator, gan_model, dataset, 
          noise=NOISE_SIZE, n_epochs=1000, n_batch=10):
    batchs_per_epoch = int(dataset.shape[0] / n_batch)
    batch_size_r=int(int(n_batch / 2))
    #generating fixed noise for examples images
    fixed_noise = generate_latent_points(OUTPUT * OUTPUT, noise)
    #fixed_noise = default_rng().normal(
    #    loc=0.0, scale=1.0, size=(OUTPUT * OUTPUT, noise))
    i=0
    work=True
    while i <= n_epochs and work:
        for j in range(batchs_per_epoch):
            #geting and generating samples
            X_real, y_real = get_real_samples(dataset, batch_size_r)
            X_fake, y_fake = generate_fake_samples(
                generator, noise, n_batch-batch_size_r)
            #joining the real images with false ones
            #X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            X_gan = generate_latent_points(n_batch, noise)
            y_gan = ones((n_batch, 1))
            y_gan = smooth_positive_labels(y_gan)
            #y_gan = noisy_labels(y_gan, 0.05)
            #training
            d_loss_r, acc_real = discriminator.train_on_batch(X_real, y_real)
            d_loss_f, acc_fake = discriminator.train_on_batch(X_fake, y_fake)
            #d_loss, acc_fake = discriminator.train_on_batch(X, y)
            #_, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            #STATS.append([i+1,(i)*batchs_per_epoch+(j+1), d_loss_r, d_loss_f, g_loss, acc_real, acc_fake])
            STATS.append([i+1,(i)*batchs_per_epoch+(j+1), d_loss_f,d_loss_r, g_loss, acc_real, acc_fake])
            print('Epoch:%d, Batch:%d/%d, d_loss_r=%.4f, g_loss=%.4f' %(i+1, j+1, batchs_per_epoch, d_loss_r, g_loss))
        if (i+1) % SAVE_FREQ == 0:
            #saving images based on fixed noise
            save_images(i+1,generator, fixed_noise)
            #save_output()
            clear_output()
            print('Accuracy on real: %.0f%%, on fake: %.0f%%' % (acc_real*100, acc_fake*100))
            #getting accuracy
            summarize_performance_2(i+1,generator,STATS)
            #acc_real,acc_fake=summarize_performance(i+1, generator, discriminator, dataset, noise)
            #checking stopping conditions
            #work=check_conditions(acc_real,acc_fake)
            #increase the real batch part on 5
            if batch_size_r<n_batch:
                batch_size_r+=int(int(n_batch / 40))            
        i+=1
            

In [ ]:
image_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
discriminator = build_discriminator(image_shape)
generator = build_generator(NOISE_SIZE, IMG_CHANNELS)
STATS=[STATS[0]]
gan_model = build_gan(generator, discriminator)
#print(train_data.shape)
train(generator, discriminator, gan_model, np.array(train_data), NOISE_SIZE,EPOCHS)
#print(train_data)

Plotting

In [ ]:
plt.rcParams["figure.figsize"] = [10, 5]
plt.rcParams["figure.autolayout"] = True
col = AVERAGE[0]
df = pd.read_csv("output\\statistics\\stat_avg_cub_0.7.csv", usecols=col,sep=', ',engine='python')
#print("Contents in csv file:", df)
plt.plot(df[[col[0]]], df[[col[1]]],color='b', label='Dyskryminator')
plt.plot(df[[col[0]]], df[[col[2]]],color='g', label='Generator')
plt.title("Accuracy during traning")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [10, 5]
plt.rcParams["figure.autolayout"] = True
col = LOSS[0]
df = pd.read_csv("output\\statistics\\stat_loss_cub_0.7.csv", usecols=col,sep=', ',engine='python')
#print("Contents in csv file:", df)
plt.plot(df[[col[1]]], df[[col[2]]],color='b', label='Dyskryminator')
plt.plot(df[[col[1]]], df[[col[3]]],color='g', label='Generator')
plt.title("Loss during traning")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.legend()

plt.show()

Generating image grid on already trained model

In [ ]:
def get_gen_model(filename=""):
    generator=Sequential()
    if(filename!=""):
        generator=load_model("models\\"+filename,compile=False)
    return generator

In [ ]:
def gen_image_grid(generator, x=1, y=1):
    latent_points = default_rng().normal(0.0, 1.0, (x*y, NOISE_SIZE))
    X = generator.predict(latent_points)
    array=np.empty(((IMG_SIZE,IMG_SIZE,IMG_CHANNELS)),float)
    count=0
    for i in range(y):
        for j in range(x):
            if j==0:
                array_pom=np.array(X[count].reshape(64,64,3,order='C'))
            else:
                array_pom=np.concatenate((
                    array_pom,np.array(X[count].reshape(
                        64,64,3,order='C'))),axis=1)
            count+=1
        if i==0:
            array=array_pom
        else:
            array=np.concatenate((array,array_pom),axis=0)
    array=(array+1)*255/2
    return array.astype(np.uint8)

In [ ]:
model = get_gen_model("gen_model_v_0.3_e_63.h5")
x=8
y=8
array = gen_image_grid(model,x,y)
image = Image.fromarray((array).astype(np.uint8))
image.show()
#image.save("output\\a1.png")